<a href="https://colab.research.google.com/github/Srikar888/Meta-Hackathon-Regional-Language-Model/blob/main/Regional_Language_Implementation_llama3_2_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Packages

In [2]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 22.9 MB/

### Import Libraries

In [3]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
GROQ_API_KEY = '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'

# GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Data Ingestion

In [5]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["/content/rtiact-kannada.pdf"])
documents = reader.load_data()

https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

In [6]:
len(documents)

30

In [7]:
documents[0].metadata
documents[1].metadata

{'page_label': '2',
 'file_name': 'rtiact-kannada.pdf',
 'file_path': '/content/rtiact-kannada.pdf',
 'file_type': 'application/pdf',
 'file_size': 763280,
 'creation_date': '2024-10-20',
 'last_modified_date': '2024-10-20'}

### Chunking

In [8]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/30 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [9]:
len(nodes)

94

In [10]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'rtiact-kannada.pdf',
 'file_path': '/content/rtiact-kannada.pdf',
 'file_type': 'application/pdf',
 'file_size': 763280,
 'creation_date': '2024-10-20',
 'last_modified_date': '2024-10-20'}

https://chunkviz.up.railway.app/

### Embedding Model

In [11]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://huggingface.co/spaces/mteb/leaderboard

### Define LLM Model

In [12]:
llm = Groq(model="llama-3.2-1b-preview", api_key=GROQ_API_KEY)

https://console.groq.com/docs/models

https://console.groq.com/keys

### Configure Service Context

In [13]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

### Create Vector Store Index

In [14]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/30 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/94 [00:00<?, ?it/s]

https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/

#### Persist/Save Index

In [15]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

#### Define Storage Context

In [16]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

https://docs.llamaindex.ai/en/stable/api_reference/storage/storage_context/

#### Load Index

In [17]:
index = load_index_from_storage(storage_context, service_context=service_context)

### Define Query Engine

In [18]:
query_engine = index.as_query_engine(service_context=service_context)

https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/

#### Feed in user query

https://docs.llamaindex.ai/en/stable/examples/prompts/prompts_rag/#viewingcustomizing-prompts

In [19]:
query = "ಬ್ರಹ್ಮ॑ಣಾಂ ಬ್ರಹ್ಮಣೋ ಬ್ರಹ್ಮಣೋ॒ ಬ್ರಹ್ಮ॑ಣಾಂ॒ ಬ್ರಹ್ಮ॑ಣಾಂ ಬ್ರಹ್ಮಣಸ್ಪತೇ ಪತೇಬ್ರಹ್ಮ"
resp = query_engine.query(query)

In [20]:
def check_verification(user_input):
    print('User Verified')
    print('☑️')


print(resp.response)
print(query)

user_input = input("Is this response satisfactory ?")
if user_input.lower() == "yes":
  check_verification(user_input)
else:
  print("Not User Verified")




ಬ್ರಹ್ಮಣಾಂ ಬ್ರಹ್ಮಣೋ ಬ್ರಹ್ಮಣಸ್ಪತೇ ಪತೇಬ್ರಹ್ಮ
ಬ್ರಹ್ಮ॑ಣಾಂ ಬ್ರಹ್ಮಣೋ ಬ್ರಹ್ಮಣೋ॒ ಬ್ರಹ್ಮ॑ಣಾಂ॒ ಬ್ರಹ್ಮ॑ಣಾಂ ಬ್ರಹ್ಮಣಸ್ಪತೇ ಪತೇಬ್ರಹ್ಮ
Is this response satisfactory ?yes
User Verified
☑️


In [21]:
def compare_strings(str1, str2):

    matches = 0

    for i in range(min(len(str1), len(str2))):

        if str1[i] == str2[i]:

            matches += 1



    percentage = (matches / min(len(str1), len(str2))) * 1000

    return percentage

In [22]:
accuracy = compare_strings(query, resp.response)
print(f"Accuracy: {accuracy}%")

Accuracy: 146.34146341463415%


In [23]:
print(resp.response)

ಬ್ರಹ್ಮಣಾಂ ಬ್ರಹ್ಮಣೋ ಬ್ರಹ್ಮಣಸ್ಪತೇ ಪತೇಬ್ರಹ್ಮ


https://itsjb13.medium.com/building-a-rag-chatbot-using-llamaindex-groq-with-llama3-chainlit-b1709f770f55

https://docs.llamaindex.ai/en/stable/optimizing/production_rag/